# Named Entity Recognition with Pretrained Model

In [1]:
import os
import wget
import pytorch_lightning as pl
from nemo.collections.nlp.models import TokenClassificationModel


ModuleNotFoundError: No module named 'nemo'

In [ ]:
# list available pre-trained models
for model in TokenClassificationModel.list_available_models():
    print(model)

ModuleNotFoundError: No module named 'nemo'

In [ ]:
# download and load the pre-trained BERT-based model
pretrained_ner_model=TokenClassificationModel.from_pretrained("ner_en_bert")

In [ ]:
# define the list of queries for inference
queries=[
    'we bought four shirts from the nvidia gear store in santa clara.',
    'Nvidia is a company.',
]

# make sample predictions
results=pretrained_ner_model.add_predictions(queries)

# show predictions
for query, result in zip(queries, results):
    print(f'Query : {query}')
    print(f'Result: {result.strip()}\n')
    print()

In [ ]:
output_dir = 'output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# evaluate model performance on sample
pretrained_ner_model.evaluate_from_file(
    text_file=os.path.join(Data, 'sample_text_dev.txt'),
    labels_file=os.path.join(Data, 'sample_labels_dev.txt'),
    output_dir=output_dir,
    add_confusion_matrix=True,
    normalize_confusion_matrix=True,
    batch_size=1
)

In [ ]:
# setup the data dir to get class weights statistics
pretrained_ner_model.update_Data(Data)

# setup train and validation Pytorch DataLoaders
pretrained_ner_model.setup_training_data()
pretrained_ner_model.setup_validation_data()

In [ ]:
# set up loss
pretrained_ner_model.setup_loss()

In [ ]:
# create a PyTorch Lightning trainer and call `fit` again
fast_dev_run=True
trainer=pl.Trainer(devices=1, accelerator='gpu', fast_dev_run=fast_dev_run)
trainer.fit(pretrained_ner_model)

In [ ]:
# evaluate model performance on sample
pretrained_ner_model.evaluate_from_file(
    text_file=os.path.join(Data, 'sample_text_dev.txt'),
    labels_file=os.path.join(Data, 'sample_labels_dev.txt'),
    output_dir=WORK_DIR,
    add_confusion_matrix=True,
    normalize_confusion_matrix=True,
    batch_size=1
)